## TV Rating Analysis on Korea


#### First step : Collect Data from Nielsen Korea

Nielsen Korea publish the top 20 TV programs by ratings on Korea every day.  
The data is updated every day. (www.nielsenkorea.co.kr)

We can collect the data from Nielsen Korea website with requests and BeautifulSoup.

In [ ]:
# import libraries
from bs4 import BeautifulSoup
import requests


- Nielsen's web page shows data by calling url ("http://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1") 
- with sub_menu(tv category), area, and date as parameters.
- We can get the data by using requests and BeautifulSoup.

In [3]:
# set URL and parameters for sample data
url = f"http://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1"
url += f"&sub_menu=1_1"  # terrestrial TV
url += f"&area=01" # GSE(Greater Seoul Area)
url += f"&begin_date=20260207" # Date

![Nielsen Korea Webpage](./images/webpage.jpg "Nielsen Korea")

In [8]:
# get page and parse html with BS4
response = requests.get(url)        
parsed_html = BeautifulSoup(response.content.decode(
    'utf-8', 'replace'), 'html.parser')


![Data Table](./images/data_table.jpg "Data Table on the page")

In [9]:
# find table and get rows for table
ranking_table = parsed_html.body.find_all(
    'table', attrs={'class': 'ranking_tb'})

rows = ranking_table[0].find_all('tr', attrs={'class': None})


In [ ]:
# check the first row => title row
rows[0]

<tr>
<td class="tb_title" colspan="4"> 가구시청률 TOP 20 </td>
</tr>

In [19]:
# check the second row => show additional information for table (area, unit)
rows[1]

<tr>
<td align="right" class="txt_9pt" colspan="4">
                
                                (분석기준: 수도권, 가구, 단위:%)
                            

                </td></tr>

In [21]:
# from the thirds row, it has the data we need
# Rank / Channel / Program / Rating
rows[2]

<tr>
<td class="tb_txt_center">1	</td>
<td class="tb_txt_center">KBS2	</td>
<td class="tb_txt">KBS2주말드라마(사랑을처방해드립니다)	</td>
<td align="center" class="percent">
                        14.7                   
                        </td>
</tr>

In [ ]:
# show all ratings data on table
i = 0
for row in rows:
    i += 1
    if (i <= 2):
        # skip headers
        continue

    # extract row
    cells = row.find_all('td')
    rank = cells[0].text.strip()
    channel = cells[1].text.strip()
    programme = cells[2].text.strip()
    rating = cells[3].text.strip()

    # convert data to dictionary
    item = {
        'rank': int(rank),
        'channel': channel,
        'rating': float(rating),
        'programme': programme,
    }

    print(
        f"{rank} {channel} {programme} {rating}")


1 KBS2 KBS2주말드라마(사랑을처방해드립니다) 14.7
2 MBC MBC금토드라마(판사이한영) 12.1
3 KBS2 KBS2토일미니시리즈(은애하는도적님아) 6.8
4 MBC MBC뉴스데스크 5.8
5 KBS2 불후의명곡 5.3
6 KBS2 살림하는남자들 4.6
7 KBS1 KBS9시뉴스 4.4
7 MBC 놀면뭐하니 4.4
9 KBS1 동네한바퀴 4.3
10 KBS1 시니어토크쇼황금연못 4.1
11 KBS1 KBS뉴스(09:30) 3.9
12 MBC 전지적참견시점 3.8
13 SBS SBS8뉴스 3.7
14 SBS 그것이알고싶다 3.6
15 KBS1 걸어서세계속으로 3.5
15 KBS1 남북의창 3.5
17 KBS1 특파원보고세계는지금 3.2
17 MBC MBC금토드라마(판사이한영<재>) 3.2
19 KBS1 KBS뉴스광장2부 3.1
19 KBS1 KBS뉴스(19:00) 3.1


In [24]:
# make a get ratings functions for multiple use
def get_ratings(date, category, area, test=False):
    """ Get ratings from nielsen Korea """
    """ Returns top rank programs for date, category, area """
    """ date : string (YYYYMMDD) """
    """ category : integer, Channel category(1: terrestrial, 3: cable, 2: general) """
    """ area : integer, Area Code (0: national wide, 1: Great Seoul Area) """

    url_string = f"http://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1"
    url_string += f"&sub_menu={category}_1"
    url_string += f"&area={area}&begin_date={date}"

    try:
        response = requests.get(url_string)
        parsed_html = BeautifulSoup(response.content.decode(
            'utf-8', 'replace'), 'html.parser')
        ranking_tb = parsed_html.body.find_all(
            'table', attrs={'class': 'ranking_tb'})

        rows = ranking_tb[0].find_all('tr', attrs={'class': None})
    except:
        print("Eror happend while accessing data on Nielsen. : ", date, category, area)
        return []

    result = []
    i = 0
    for row in rows:
        i += 1
        if (i <= 2):
            # skip headers
            continue

        # extract row
        items = row.find_all('td')
        rank = items[0].text.strip()
        channel = items[1].text.strip()
        programme = items[2].text.strip()
        rating = items[3].text.strip()

        # convert data to dictionary
        item = {
            'rank': int(rank),
            'channel': channel,
            'rating': float(rating),
            'programme': programme,
            'date': date[:4] + '-' + date[4:6] + '-' + date[6:8]
        }

        if test:
            print(
                f"TEST MODE : {date} {rank} {channel} {programme} {rating}")

        result.append(item)

    return result


#### COLLECT DATA

Let's collect 2025 data from Nielsen Korea.

In [25]:
days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
results = []

for month in range(1, 13):
    print(month)
    for day in range(1, days[month-1]+1):
        date = f"2025{month:02d}{day:02d}"
        ratings = get_ratings(date, 1, 1)
        results += ratings

print("DONE : ", len(results))


1
2
3
4
5
6
7
8
Eror happend while accessing data on Nielsen. :  20250817 1 1
Eror happend while accessing data on Nielsen. :  20250818 1 1
9
10
11
12
DONE :  7260


In [27]:
# save data with pickle
import pickle
with open('tvratings_2025.pkl', 'wb') as f:
    pickle.dump(results, f)

#### Let's analyze the most watched TV programs in Korea, 2025.

In [28]:
# import libraries for data analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [30]:
# make a dataframe with results
df = pd.DataFrame(results)

In [ ]:
# check the data
df.head()

,rank,channel,rating,programme,date
0,1,KBS1,11.4,일일드라마(결혼하자맹꽁아),2025-01-01
1,2,MBC,10.7,MBC뉴스데스크,2025-01-01
2,3,KBS2,8.6,일일드라마(신데렐라게임),2025-01-01
3,4,KBS1,6.8,KBS9시뉴스,2025-01-01
4,5,KBS1,6.6,인간극장,2025-01-01


In [33]:
# check data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7260 entries, 0 to 7259
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   rank       7260 non-null   int64  
 1   channel    7260 non-null   object 
 2   rating     7260 non-null   float64
 3   programme  7260 non-null   object 
 4   date       7260 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 283.7+ KB


In [34]:
# describe the data
df.describe()

,rank,rating
count,7260.000000,7260.000000
mean,10.297521,4.779656
std,5.663077,2.495339
min,1.000000,1.700000
25%,5.000000,3.100000
50%,10.000000,4.000000
75%,15.000000,5.700000
max,20.000000,20.300000
